In [ ]:
install.packages("devtools")
devtools::install_github("gadenbuie/rsthemes")
rsthemes::install_rsthemes()
rstudioapi::applyTheme("Oceanic Plus {rsthemes}")

In [ ]:
# Packages 

install.packages("ggridges")
install.packages("corrr")
install.packages("glmnet")
install.packages("xgboost")

library(tidyverse)
library(skimr)
library(janitor)
library(ggridges)
library(corrr)
library(tidymodels)
library(kableExtra)
library(patchwork)

# Helpers
knitr::opts_chunk$set(message = FALSE,
                      warning = FALSE,
                      echo = FALSE)

theme_set(theme_minimal())+
  theme_update(axis.title = element_blank())

## Problem definition

The first step in this analysis is to understand and define the problem that we are trying to solve. It has been mentioned that at Discount Motors (a used car dealership in the UK ) the recently hired junior sales people have difficulties pricing used cars that arrive at the dealership.

This has caused sales to decline 18% in recent months,so in this analysis we are going to develop a machine learning model to price cars that arrive with more precision. It is known that cars that are more than £1500 above the estimated price will not sell. So the intention is to keep the model predictions within this range as much as possible.

## Data

We are going to work with the Toyota specialist to test this solution approach. The specialists have collected data from other retailers on the price that a range of Toyota cars were listed at.

A data description is presented here:

| Feature      | Description                                    |
|--------------|------------------------------------------------|
| model        | Model name of the Toyota car                   |
| transmission | Transmission type of the car                   |
| fuelType     | The fuel type the car engine uses              |
| year         | Year when the car was launched                 |
| price        | Price of the car from retailers                |
| mileage      | Number of miles that the car has been traveled |
| tax          | Sales tax of the car                           |
| mpg          | Miles traveled per gallon of fuel consumed     |
| engineSize   | Size of the engine                             |

: Table 1: Features description

A summary and a first look at the data is also presented:

In [ ]:
toyota <- read_csv("data/toyota.csv")

toyota %>% 
  skim_without_charts() %>% 
  kbl() %>% 
  kable_styling()

toyota %>% 
  slice_sample(n = 10) %>% 
  kbl() %>% 
  kable_styling()


# Changing column names to snake case

toyota <- toyota %>% 
  clean_names()

If we look at the summary, we notice that:

- There are not missing data.
- We have numerical and categorical variables.
- There are not clear data inconsistencies at the moment.

The only cleaning step we perform is to change the column names into a snake case format (i.e. column_name ) to keep simplicity in performing the analysis.

Now that we have defined the problem, looked the description of our data and made a  initial sanity check of our data its time to start with the Exploratory Data Analysis (EDA).

## Exploratory data analysis

If we analyze the type of problem we are dealing with, we conclude:

- The objective is to predict the sale price from different characteristics of the used car like model, the year of the release, the miles traveled, fuel type, transmission etc;

- So we have a regression problem, our outcome is the price and the predictors are the features mentioned above, the prediction must be as precise as possible with the data we have.

It is known by theory that the next features are possibly related with the price of a car

- Years: With more years since the release of the car, is possible that the price is lower.

- Model: Sport models may have higher prices than other models

- The relation with other variables is not clear, so we need to explore it.

In order to do so, we analyze some of the features of the data to accept or reject the prior information we have and determine what features are going to be included in the model. 

In [ ]:

# Helper function to plot numerical and categorical vars

plot_num_cat <- function(tbl,num,cat,type){
  if (type == "box") {
    tbl %>% 
      ggplot(aes(x = fct_reorder({{cat}},{{num}}),
                 y = {{num}}))+
      geom_boxplot()+
      coord_flip()
  }
  else if(type == "violin"){
    tbl %>% 
      ggplot(aes(x = {{num}},
                 y = fct_reorder({{cat}},{{num}})))+
      geom_violin()
  }
  else if(type == "ridges"){
    tbl %>% 
      ggplot(aes(x = {{num}},
                 y = fct_reorder({{cat}},{{num}})))+
      stat_density_ridges(quantile_lines = T)
  }
}


### Distribution of price

Firstly lets take a look at the distribution of the price:

In [ ]:
# No transformation
p1 <- toyota %>% 
  ggplot(aes(x = price))+
  geom_histogram(color = "white")+
  ggtitle("Distribution of price",
          subtitle = "No transformation")

# Log transformation on price 
p2 <- toyota %>% 
  ggplot(aes(x = price))+
  geom_histogram(color = "White")+
  scale_x_continuous(trans = "log10")+
  ggtitle("Distribution of price",
          subtitle = "log10 transformation")

(p1+p2)+
  plot_annotation(caption = "Plot 1: Comparing distribution of price")

The distribution of price looks clearly not normal, so a log 10 transformation may be needed in order to get more accurate results, so now on we are going to work with the log of the price.

In [ ]:
# Creating log 10 of price

toyota <- toyota %>% 
  mutate(price_log10 = log10(price))

### Comparing price across categories

In this section we are going to compare the price to different categorical variables to identify patterns that should be captured by our model.

#### Price and the diferent car models

In [ ]:
toyota %>% 
  plot_num_cat(price,model, type = "ridges")+
  scale_x_log10()+
  theme(axis.title.x = element_text())+
  labs(x = "Price (log 10 scale)")+
  ggtitle("Prices by type of model")

If we look at the plot, we can spot a difference in the price range distribution depending on the model of the car, the "Supra" a sport Toyota car has a higher median price than other more urban cars like "Yaris". So the model of the car should be considered as a feature of our model.

### Price and transmission

In [ ]:
toyota %>% 
  tabyl(transmission) %>%
  kbl() %>% 
  kable_minimal(full_width = F) %>%
  footnote(general = "Table 2: Count of transmission type")
  
  

toyota %>%
  filter(transmission != "Other") %>% 
  plot_num_cat(price,transmission, type = "box")+
  scale_y_log10()+
  theme(axis.title.x = element_text())+
  labs(y = "Price",
       caption = "Plot 2: Comparing the distribution of transmission type")+
  ggtitle("Price by transmission type")

If we look at Table 2 we see that the transmission type other and semi-auto are under represented in comparison to automatic and manual transmission groups, so we should consider make just one level for the under represented transmission types.

Now, if we observe now at Plot 2 we can make the following insights:

-   Higher prices are associated to automatic cars
-   The difference between semi-auto and manual cars is not that clear.

### Price and fuel type

In [ ]:
toyota %>% 
  tabyl(fuel_type) %>% 
  kbl() %>% 
  kable_minimal(full_width = F) %>%
  footnote(general = "Table 3: Count of fuel type")

toyota %>%
  plot_num_cat(price,fuel_type,type = "box")+
  scale_y_log10()+
  theme(axis.title.x = element_text())+
  labs(y = "Price",
       caption = "Plot 3: Price by fuel type")+
  ggtitle("Comparing price for different fuel types")

Looking at Plot 3 clearly there is a pattern, hybrid type cars have a higher median price than diesel and petrol,then the type of fuel should be considered in our model.

Notice that the groups of diesel and other are under represented in the data set in comparison of the hybrid and diesel groups, so like the model variable we could consider group other and diesel together.

### Prices across the diferent years

In [ ]:
toyota %>% 
  group_by(year) %>% 
  summarise(median_price = median(price),
            sd = sd(price)) %>% 
  filter(year>2002) %>% 
  ggplot(aes(x = year, y = median_price))+
  geom_line()+
  geom_point()+
  geom_errorbar(aes(ymin=median_price-sd, ymax=median_price+sd), width=.2,
                 position=position_dodge(0.05))+
  scale_y_log10()+
  theme(axis.title.y = element_text())+
  labs(y = "Price (log 10 scale)",
       caption = "Plot 3. Median prices by year (with error bars showing the dispersion of the distribution)")+
  ggtitle("Comparing median prices by year")



As expected we spot a positive trend, the median prices have been increasing as the years pass so the year of the car is an important feature in our model.

## Comparing prices across numerical variables

The next step in our analysis is to determine what numerical variables could be useful predictor for the price of the car.

### Correlations across numerical variables

Lets look at the correlation of the numerical variables:

In [ ]:
cor <- toyota %>% 
  select_if(is.numeric) %>% 
  select(-price) %>% 
  correlate() %>% 
  arrange()

cor %>% 
  kbl() %>% 
  kable_minimal(full_width = F) %>%
  footnote(general = "Table 4: Correlations for numerical vars")
  
cor %>% 
  network_plot(repel = T)

If we take a look at the correlation of the variables we conclude that:

- The engine size could provide some information about the price, so its worth to analyze their scatter plot
- The tax and miles per gallon provide almost no information about the price, so they wont be considered in the model
- Price and mileage provide useful information about the price, again we will analyze their scatter plot.

### Prices and mileage

In [ ]:
toyota %>% 
  ggplot(aes(x = mileage, y = price))+
  geom_point(alpha = 0.3, size = 0.5)+
  scale_y_log10()+
  stat_smooth(method = "lm")+
  theme(axis.title = element_text())+
  labs(y = "Price (log 10 scale)",
       caption = "Plot 5: Comparing price and mileage")+
  ggtitle("Price vs Mileage")

As expected there is a clear negative relationship in the number of miles traveled with the car price

-   On average, the higher the mileage the lower the price.

But notice from the correlation table that the two variables (year and mileage) are capturing the same information. So it would be a good practice to drop one of this correlated variables when making the model.

### Prices and engine size.

In [ ]:
toyota %>% 
  filter(engine_size > 0, engine_size < 4.2) %>% 
  mutate(engine_size = as.character(engine_size)) %>% 
  ggplot(aes(x = fct_reorder(engine_size,price),
             y = price))+
  geom_boxplot()+
  scale_y_log10()+
  theme(axis.title = element_text())+
  labs(y = "Price (log 10 scale)",
       x = "Engine size",
       caption = "Plot 6: Comparing prices and engine size")+
  ggtitle("Prices by engine size")

There is a clear pattern higher engine_size, then higher the price of the car on average (engine size of zero and less than 4.2 were omitted because they have only one observation). The engine size variable should be considered in out model.

## Data modeling

With the observed in the exploratory data analysis (EDA) we conclude that the most important variables that affect in the log price are:

-   Year
-   Mileage
-   Engine size
-   Model
-   Transmission
-   Fuel type

But, we notice earlier that year and mileage essentially capture the same information so we will not consider the mileage. And we wont consider interactions between predictors for simplicity.

As mentioned earlier given that we want to predict the log of the price when a new car arrives based on numerical and categorical variables we have a regression problem. And we are going to compare tree types of models that are suitable for regression tasks:

- Linear Regression: A traditional, simpler model.
- Regularized lasso regression: Since we have a lot of categorical predictors and it may be a best option to let the model "choose" the relevant predictors.
- Boosted Trees (XGBoost): To make a more robust model, and possibly improve prediction capabilities.

Remember that when we start our analysis we said that cars that are more than £1500 above the estimated price will not sell. So the intention is to keep the model predictions within this range as much as possible, we are going to do this by choosing the rmse (Root Mean Squared Error) as our metric to compare the tree models. The lower the rmse, better precision of our predictions.

### Spending data budget

We split our data stratifying by Price (wich means that we want to keep almost the same distribution of prices in the train and test sets) keeping 75% of our data to estimate the parameters of our model and 25% to test the model on data that has never seen to see how well it would perform on new data.

Lets take a look at our training and testing data sets

In [ ]:
# Setting a seed for reproducibility
set.seed(123)

spl <- toyota %>%
  initial_split(strata = price_log10)

# Train set
train <- spl %>% 
  training()

train %>% 
  slice_sample(n = 6) %>% 
  kbl() %>% 
  kable_minimal(full_width = F) %>%
  footnote(general = "Table 5: Training set")

  
# Test set
test <- spl %>% 
  testing()

test %>% 
  slice_sample(n = 6) %>% 
  kbl() %>% 
  kable_minimal(full_width = F) %>%
  footnote(general = "Table 6: Test set")

We are going to estimate the model parameters with ~ 5000 observations and testing it on ~ 1600 observations.

To validate the model we are going to use a 10-fold cross validation.

In [ ]:
set.seed(456)

folds <- vfold_cv(train, strata = price_log10,v = 10)

### Model selection

Before actually train and validate our models we need to do the next pre-procesing steps and feature engenieering, according to what we observed in the EDA:

- Drop unusual values for the engine size variable (engines with size zero and groups of engine size of only one observation).
- Group unrepresented categories for transmission, fuel type and also by model to reduce the number of predictors.
- Make dummy variables.

In [ ]:
# Pre-procesing steps, feature engineering
base_rec <- recipe(price_log10 ~ 
                        year + engine_size + model+
                        transmission + fuel_type,
                      data = train) %>% 
  step_other(transmission,fuel_type,model,
             threshold = .05) %>% 
  step_filter(engine_size > 0, engine_size < 4.2) %>% 
  step_dummy(all_nominal_predictors())

### Model specification

- For linear regression: A usual linear regression model
- For lasso regression: Here we want to determine the best value for the penalty, that indicate what features are going to have more impact on the model. 
- For boosted trees: Here we want to determine the number of trees to have the lower value of rmse possible.


In [ ]:
# Model specifications

lin_reg_spec <- linear_reg() %>% 
  set_mode("regression") %>% 
  set_engine("lm")

lasso_reg_spec <- linear_reg(penalty = tune(),
                             mixture = 1) %>%
  set_mode("regression") %>% 
  set_engine("glmnet")

xgb_spec <- boost_tree(trees = tune()) %>% 
  set_mode("regression") %>% 
  set_engine("xgboost")

### Model validation

We are going to determine the parameters for the penalty (lasso regression) and the number of trees (boosted trees) using a grid search among the possible values in the parameter space.

In [ ]:
# workflows
lin_reg_wf <- workflow(base_rec,lin_reg_spec)
lasso_reg_wf <- workflow(base_rec, lasso_reg_spec)
xgb_wf <- workflow(base_rec,xgb_spec)

# grids for parameter search
lasso_reg_grid <- grid_regular(penalty(
  range = c(-5,0)),
  levels = 10)

xgb_grid <- grid_regular(
  trees(c(500,2000)),
  levels = 10
)

# metric set
m_set <- metric_set(rmse)

Now we train the model in 10 different subsets of our training data and validating in 10 other different subsets of our training data to see how would the model perform in the test data (data that has never seen before).

We observe the next results:

In [ ]:
# Model validation

# Linear regression
lin_reg_res <- lin_reg_wf %>% 
  fit_resamples(resamples = folds,
                metrics = m_set)

lin_reg_metrics <-lin_reg_res %>% 
  collect_metrics() %>% 
  select(.metric,mean,std_err) %>% 
  mutate(model = "linear_regression")

# Lasso regression
set.seed(789)

lasso_reg_res <- tune_grid(
  lasso_reg_wf,
  resamples = folds,
  metrics = m_set,
  grid = lasso_reg_grid
)


# Selecting a simpler model with the best rmse

lasso_reg_metrics <-
  lasso_reg_res %>% 
  select_by_one_std_err(metric = "rmse", desc(penalty)) %>%
  select(.metric,mean,std_err) %>% 
  mutate(model = "lasso_regression")

# XGBoost

xgb_res <- tune_grid(
  xgb_wf,
  resamples = folds,
  metrics = m_set,
  grid = xgb_grid
)

xgb_metrics <- xgb_res %>% 
  show_best(n = 1) %>% 
  select(.metric,mean,std_err) %>% 
  mutate(model = "XGBoost")

model_comp <- lin_reg_metrics %>% 
  bind_rows(lasso_reg_metrics) %>% 
  bind_rows(xgb_metrics)

In [ ]:
model_comp %>% 
  kbl() %>% 
  kable_minimal(full_width = F) %>%
  footnote(general = "Table 6: Model results comparision")
  
model_comp %>%
  ggplot(aes(y = mean, x = model, color = model))+
  geom_point()+
  geom_errorbar(aes(ymin=mean-std_err,
                    ymax=mean+std_err),
                width=.2,
                position=position_dodge(0.05))+
  theme(legend.position = "none",
        axis.title.y = element_text())+
  labs(y = "rmse",
       caption = "Plot 7: Comparing models (with error bar for the estimators")+
  ggtitle("Comparing model validation performance")

Despite the linear regression preformed even better than lasso regression, since we want prediction power we will keep with the XGBoost model.

### Model selection

Now lets see of the XGBoost can be further improved increasing the number of trees search (i.e. making a bigger grid for the parameter).

In [ ]:
# Tuning the numer of trees

xgb_tune_spec <- boost_tree(trees = tune()) %>% 
  set_mode("regression") %>% 
  set_engine("xgboost")

# workflow
xgb_tune_wf <- workflow(base_rec,xgb_tune_spec)

# bigger grid
xgb_tune_grid <- grid_regular(trees(),
                              levels = 15)

# validation (it may take some minutes)
set.seed(1011)

xgb_tune_res <- tune_grid(
  xgb_tune_wf,
  resamples = folds,
  metrics = m_set,
  grid = xgb_tune_grid
)


In [ ]:
xgb_tune_res %>% 
  show_best() %>% 
  select(trees,.metric,mean,std_err) %>% 
  kbl() %>% 
  kable_minimal(full_width = F) %>%
  footnote(general = "Table 7: XGBoost comparision")

We see that there are no further improvements in the rmse metric above the ~140 trees so we should keep the simpler model.

> So the final model selection is an XGBoost with 143 trees.

### Modeling results

We now fit the selected model to all the training test and see how well it performed on data that has never seen before.

In [ ]:
final_xgb <- xgb_tune_res %>% 
  select_best()

final_res <- xgb_tune_wf %>% 
  finalize_workflow(final_xgb) %>% 
  last_fit(spl)

collect_metrics(final_res) %>% 
  kbl() %>%
  kable_minimal(full_width = F) %>%
  footnote(general = "Table 8: XGBoost on new data")

We now see that our XGBoost model performed in the test set (0.5718 RMSE) almost as good as in the training set (0.5211 RMSE).

Now the final step is to check that this models predictions on new data beat the benchmark we set before: the predictions should not be in a range of £1500 above the estimated price or the car will not sell.

But remember that we predicted values for the log price, so we need to take back the predicted values to the original space using the inverse transformation.

In [ ]:
benchmark <-
  collect_predictions(final_res) %>% 
  select(.pred) %>% 
  rename(pred_log10 = .pred) %>% 
  bind_cols(
    test %>% 
      select(price,price_log10)
  ) %>% 
  mutate(pred = 10^pred_log10,
         diff = price-pred,
         is_in_range = ifelse(diff>=-1500,1,0))

benchmark %>% 
  ggplot(aes(x = pred, y = price))+
  geom_point(alpha = 0.8, size = 0.8)+
  geom_abline(slope = 1, intercept = 0, color = "steelblue")+
  theme(axis.title = element_text())+
  labs(caption = "Plot 8: Comparing model actual prices vs predicted on the test set")+
  ggtitle("Actual prices vs Predicted")

benchmark %>% 
  tabyl(is_in_range) %>%
  kbl() %>%
  kable_minimal(full_width = F) %>%
  footnote(general = "Table 8: Are the predictions in the desired range ? ")

If we look at Plot 8 we have a better sense on how the model performed on new data, the model performed very good on new data. But note that for prices above ~  £40,000 the model has some troubles to predict new data.

Now the Table 8 indicate us that 93% percent of our predictions on new data are in a range of £1500 below the estimated price. So we can conclude that 93% of Toyota cars that arrive at the new dealerships will be sold.

In [ ]:
# Saving the model for future predictions or work

final_wf <- extract_workflow(final_res)

saveRDS(final_wf, "final_wf.rds")

## Conclusions

Recapitulating, we developed a XGBoost model for predicting prices for Toyota cars that arrive to the dealerships. This model will help the Jr. sales people to price the new cars more effectively, which means that more cars will be sold.

On this test the model correctly predicted 93% of Toyota cars that has never seen before in the desired range, making this a good indicator to use the model for other car brands, and implemented on the diferent dealerships.

## Further actions

- Despite this is a good model if our desire is to implement it on more brands, new features will be needed and a model adjust may be necessary.
- The model performance needs to be tracked over the time, and it may need adjustments due to changes in the market.
- It may worth the while to consider model interactions between predictors (which was not contemplated in this analysis)
- The stakeholder with its experience should consider tell us to incorporate more features or provide more market knowledge in order to improve the predictions and then make better data-driven decisions.
